In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'size_length_cookware'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Plates & Platters"""

regex_pattern = r'(?i)(Measures\s?\d*.?\d*.?\")|(\s\d*.?.?Inch)'
# regex_pattern = r"(?i)(\b[8][1-9] in\w* Overall Length)|(\b[9][0-9] in\w* Overall Length)|(\b1[0][0] in\w* Overall Length)"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
metric = ' Inches'

range_string = """
 0 - 2.9
 12 - 14.9
 15 - 17.9
 18 - 20.9
 21 or Greater
 3 - 5.9
 6 - 8.9
 9 - 11.9
 """

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df

product_id external_id  \
0     1185374222    69436321   
1     1213281732    69686315   
2      988484855    69597686   
3      817164556    43267903   
4      817153737    11194648   
...          ...         ...   
6244   817315484    65555163   
6245   817349161    60593665   
6246   817255607    61261631   
6247   817554834    64671512   
6248   817401290    69339110   

                                                                               product_name  \
0                   Farberware® Cookstart Nonstick 11-Inch Aluminum Square Griddle in Black   
1                                        NFL Philadelphia Eagles Aluminum Patterned Coaster   
2     Zwilling® J.A. Henckels Clad CFX Ceramic Nonstick Stainless Steel 2-Piece Fry Pan Set   
3                                         Zwilling J.A. Henckels Aurora 8-Inch Open Fry Pan   
4                                         Farberware® Classic Nonstick 12-Inch Deep Skillet   
...                                                                                     ...   
6244                         Mikasa® Swirl Ombre Square 16-Piece Dinnerware Set in Graphite   
6245                                               Beaded Charger Plates in Gold (Set of 6)   
6246                                            Kosta Boda Bruk 82.5 oz. Clear Serving Bowl   
6247                                                        Fiesta® Salad Plate in Mulberry   
6248                                    Golden Rabbit® Cobalt Swirl 15.5-Inch Round Platter   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 The Farberware Cookstart Square Griddle is constructed from heavy-duty aluminum for excellent heat distribution and durability that lasts. DiamondMax nonstick interior performs 3X better than ceramic nonstick for easy food release and cleanup. Durable, heavy-duty aluminum construction delivers excellent heat distribution DiamondMax nonstick interior coating performs 3X better than ceramic nonstick for easy food release and cleanup Dual-riveted resin handle provides a confident grip   
1                                                       

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0" - 2.9"',
 '12" - 14.9"',
 '15" - 17.9"',
 '18" - 20.9"',
 '21" or Greater',
 '3" - 5.9"',
 '6" - 8.9"',
 '9" - 11.9"',
 'n/a']

In [7]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0       []
1       []
2       []
3       []
4       []
        ..
6244    []
6245    []
6246    []
6247    []
6248    []
Name: long_desc, Length: 6249, dtype: object

In [8]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [9]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [10]:
x

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [11]:
x[0:100]

product_id external_id  \
1    1213281732    69686315   
5     817476141    62441551   
6     817476158    62441964   
7     817498227    41016350   
8     817555221    64713052   
9     817631351    69487325   
10    817636220    69517014   
11    988487679    69624176   
12    817471970    61005426   
13   1007044173    69634980   
14   1070130951    69634988   
15   1156991209    17140353   
16   1143303850    43965625   
17   1162593918    69691619   
18   1110713531    69514005   
19   1159730107    69691612   
20   1087371535    69648086   
21   1099397540    62441841   
22   1065850077    69648119   
23   1018170434    69635293   
24   1010396275    69620854   
25   1005128244    69625779   
26    997140804    67730995   
27   1005130703    69634973   
28    997149448    67898190   
29    997140805    67731008   
30    988772391    16938203   
31    988513028    46318169   
32    988667126    77477977   
33    988485219    44040031   
34    817193111    76497963   
35    817479110    62976442   
36    817479104    62976282   
39   1219063828    63034868   
40   1219063828    63034868   
41    817190531    62869126   
42    817187307    62358927   
44    817186030    62055840   
45    817185980    62018258   
46    817185587    61870994   
47    817183857    61460836   
48    817183849    61460621   
50    817471286    60782281   
51    817179496    60432186   
52    817180643    60782342   
54    817175369    47218086   
55    817204945    47170971   
56   1162894683    47147461   
60       867570    46889720   
69    817463166    45478840   
70    817201368    45583957   
72    817195077    44959944   
73    817167435    44511562   
75   1203399509    44321147   
76    817460897    43921430   
77    817200089    43626199   
78    817200064    43626083   
79    817460359    43597789   
80    817200058    43551552   
81    817165450    43781690   
82       171840    43748068   
85    817459684    43082575   
87    817458839    42427056   
88    817161944    41750254   
90    817458718    42276203   
94    817162224    41937716   
95    817162455    42076360   
96      3745823    42075790   
97      3745798    42075547   
100   817160123    40522609   
101   817160104    40518220   
103   817159633    40194158   
104   817456548    40190969   
105   817159583    40159386   
106   817482616    18882442   
108   817158173    18903520   
109   817158149    18903458   
110   817455575    18903385   
111   817158164    18901803   
112   817455563    18901706   
113   817158158    18901633   
114   817158154    18901595   
115   817455557    18901579   
116   817482594    18900467   
117   817455242    18544385   
119   817195369    18515016   
122   817454679    17551060   
125   817454126    16443352   
126   817454279    16750700   
127   817664979    16613550   
129   817453392    14272321   
130   817480290    15161710   
131   817162249    12999496   
134   817453307    14037381   
135  1195573372    14037330   
136   817155150    14554041   
137      300724    14450700   
139   817452812    12822030   
142   817154001    12402660   
153  1214062103    69705501   

                                                                              product_name  \
1                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8                                    13-Inch Scalloped Charger Plates in Silver (Set of 6)   
9                   kate spade new york Vintage Cherry Dot Tidbit Plates in Red (Set of 4)   
10                           Certified International Sweet Bunny Dessert Plates (Set of 4)   
11                  Certified International Winter Garden Dessert Plates in Red (Set of 4)   
12                         

In [12]:
# df[df['matches'].astype(str)=='[]']

In [13]:
# x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

In [14]:
# df[df['external_id'].astype(str)=='00739441']

In [15]:
# x['matches'].explode().value_counts()

In [16]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [17]:
split_col(x)

In [18]:
# x['matches'].explode().value_counts()

In [19]:
error

NameError: name 'error' is not defined

In [20]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\"\-','',str(measure)))
x

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [21]:
def remove_fractions(numbers):
    if numbers.find('/') > -1:
#         numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [30]:
# x['matches'].explode().value_counts()

In [23]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [24]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'\-','',str(measure)))
x

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [36]:
x=x[x['matches'].astype(str)!="10'"]
x=x[x['matches'].astype(str)!=""]

In [32]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'\"','',str(measure)))
x

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [37]:
x[x['matches'].astype(float)>1]

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [38]:
x['matches']=x['matches'].astype(float)

In [39]:
x

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

# matches

In [40]:
matches=x
matches

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [41]:
matches['matches']=matches['matches'].astype(float)

In [42]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 2.9:
        return " 0 - 2.9 Inches"
        
    if num >=  12 and num <= 14.9:
        return " 12 - 14.9 Inches"
        
    if num >=  15 and num <= 17.9:
        return " 15 - 17.9 Inches"
        
    if num >=  18 and num <= 20.9:
        return " 18 - 20.9 Inches"
        
    if num >=  3 and num <= 5.9:
        return " 3 - 5.9 Inches"
        
    if num >=  6 and num <= 8.9:
        return " 6 - 8.9 Inches"
        
    if num >=  9 and num <= 11.9:
        return " 9 - 11.9 Inches"
        
    else:
        return r"n/a"



In [43]:
# matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [44]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

product_id external_id  \
1     1213281732    69686315   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
8      817555221    64713052   
...          ...         ...   
6241   817392539    68958428   
6242   817382662    68671785   
6243   817341581    66760788   
6245   817349161    60593665   
6247   817554834    64671512   

                                                      product_name  \
1               NFL Philadelphia Eagles Aluminum Patterned Coaster   
5                      Vista Alegre Carré Dinner Plates (Set of 4)   
6     Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7        Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8            13-Inch Scalloped Charger Plates in Silver (Set of 6)   
...                                                            ...   
6241             Bee & Willow™ Home Plaid Appetizer Plate in Green   
6242   Lenox® Textured Neutrals™ Accent Plates in Taupe (Set of 4)   
6243           JJ Cole® Insulated Bottle Cooler Bag in Black/White   
6245                      Beaded Charger Plates in Gold (Set of 6)   
6247                               Fiesta® Salad Plate in Mulberry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
1                                                     Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
5                                                                                                                                                                                                                                                                                      The Asian-inspired Carré Dinnerware Collection from Vista Alegre brings sleek, minimalist style to your table. Beautifully crafted in white porcelain, this elegant dinnerware has a soft square shape and a simply sophisticated look. Sleek and minimalist Asian-inspired design Soft square shape Set of 4 Porcelain Microwave and dishwasher safe Measures 10.47\" sq. Imported   
6                                                                                                                                                                                                            Inspired by the natural textures of coral and shells, the Mar Bread and Butter Plate from Vista Alegre features an exquisite white-on-white pattern on fine porcelain. This lovely design is durable for everyday dining and impressive for special occasions. White-on-white pattern inspired by textures of coral and shells Set of 4 Durable for every day and special occasions Porcelain Dishwasher and microwave safe Measures 7.75\" diameter Imported   
7                                                                                                                                                                   

In [46]:
matches['Q:size_length_cookware'].explode().value_counts()

 6 - 8.9 Inches      1440
 9 - 11.9 Inches     1357
 12 - 14.9 Inches     497
 3 - 5.9 Inches       376
 15 - 17.9 Inches     129
 18 - 20.9 Inches      84
 0 - 2.9 Inches        59
n/a                    53
Name: Q:size_length_cookware, dtype: int64

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [49]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/BedBathandBeyond - {attribute} - match upload_Shawn.csv',index=False)

In [51]:
looks_good('Bed Bath & Beyond', attribute, df, matches)